In [1]:
import os
from dotenv import load_dotenv
from langchain_neo4j import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.chains import RetrievalQA


import vansy_f8a_genai_test.report_a03_rag_and_reasoning_frameworks_tutorial_prompt as prompt_hub

In [2]:
load_dotenv()

True

In [3]:
memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True
)

/var/folders/lg/yb401ry14w19vfs73n7y2rtw0000gn/T/ipykernel_5808/88119791.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


### Hospital Cypher Chain

In [4]:
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
)

graph.refresh_schema()

/var/folders/lg/yb401ry14w19vfs73n7y2rtw0000gn/T/ipykernel_5808/249957495.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [5]:
cypher_generation_template = prompt_hub.CYPHER_GENERATION_TEMPLATE
qa_generation_template = prompt_hub.QA_GENERATION_TEMPLATE

In [6]:
cypher_generation_prompt = PromptTemplate(
    input_variables=["schema", "question"], template=cypher_generation_template
)

qa_generation_prompt = PromptTemplate(
    input_variables=["context", "question"], template=qa_generation_template
)


In [7]:
hospital_cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    qa_llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    graph=graph,
    verbose=True,
    qa_prompt=qa_generation_prompt,
    cypher_prompt=cypher_generation_prompt,
    validate_cypher=True,
    top_k=100,
    allow_dangerous_requests=True,
    # use_function_response=True
)

### Hospital Review Chain

In [8]:
from langchain_neo4j import Neo4jVector
from langchain_openai import OpenAIEmbeddings

In [9]:
neo4j_vector_index = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(model="text-embedding-3-small"),
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="reviews",
    node_label="Review",
    text_node_properties=[
        "physician_name",
        "patient_name",
        "text",
        "hospital_name",
    ],
    embedding_node_property="embedding",
)

In [10]:
review_generation_template = prompt_hub.REVIEW_GENERATION_TEMPLATE

In [11]:
review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_generation_template
    )
)

In [12]:
review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)

In [13]:
messages = [review_system_prompt, review_human_prompt]
review_prompt = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

In [14]:
reviews_vector_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini", temperature=0),
    chain_type="stuff",
    retriever=neo4j_vector_index.as_retriever(k=12),
)
reviews_vector_chain.combine_documents_chain.llm_chain.prompt = review_prompt

### ReAct Agent

In [15]:
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_openai import ChatOpenAI
from langchain import hub

In [16]:
# 1. Define the tools for agent can use
tools = [
    Tool(
    name="HospitalGraphQA",
    # The 'func' is the chain's invoke method. We use a lambda to map the
    # string input from the agent to the dictionary input the chain expects.
    func=lambda q: hospital_cypher_chain.invoke({"query": q}),
    description="""Useful for answering questions about patients,
        physicians, hospitals, insurance payers, patient review
        statistics, and hospital visit details. Use the entire prompt as
        input to the tool. For instance, if the prompt is 'How many visits
        have there been?", the input should be "How many visits have
        there been?'
        """
    ),
    Tool(
    name="HospitalReviewsQA",
    func=lambda q: reviews_vector_chain.invoke({"query": q}),
    description="""Useful when you need to answer questions
        about patient experiences, feelings, or any other qualitative
        question that could be answered about a patient using semantic
        search. Not useful for answering objective questions that involve
        counting, percentages, aggregations, or listing facts. Use the
        entire prompt as input to the tool. For instance, if the prompt is
        "Are patients satisfied with their care?", the input should be
        "Are patients satisfied with their care?".
        """
    )
]

In [17]:
# 2. Set up the ReAct Agent
# Pull the standard ReAct prompt
prompt = hub.pull("hwchase17/react")

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
prompt.dict()

{'name': None,
 'input_variables': ['agent_scratchpad', 'input', 'tool_names', 'tools'],
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {},
 'metadata': {'lc_hub_owner': 'hwchase17',
  'lc_hub_repo': 'react',
  'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'},
 'tags': None,
 'template': 'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}',
 'template_format': 'f-string',
 'validate_t

In [19]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [20]:
custom_prompt_template = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

PREVIOUS CHAT HISTORY:
{chat_history}

NEW QUESTION:
Question: {input}
Thought:{agent_scratchpad}
"""

In [21]:
prompt = PromptTemplate(
    input_variables=['chat_history','agent_scratchpad', 'input', 'tool_names', 'tools'],
    template=custom_prompt_template,
)

In [22]:
# Choose the LLM for the agent to use
# This can be the same or different from the models in your chain
agent_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [23]:
# Create the agent
agent = create_react_agent(agent_llm, tools, prompt)

In [24]:
# Choose the LLM for the agent to use
# This can be the same or different from the models in your chain
agent_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [25]:
# Choose the LLM for the agent to use
# This can be the same or different from the models in your chain
agent_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [26]:
# Choose the LLM for the agent to use
# This can be the same or different from the models in your chain
agent_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [27]:
# Create the agent executor to run the agent
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, verbose=True, 
                               memory=memory, 
                               return_intermediate_steps=True,
                               handle_parsing_errors=True)

In [32]:
# --- 3. The Interactive Chat Loop ---
print("Chat with the Hospital AI Agent! Type 'exit' to quit.")
while True:
    try:
        user_question = input("You: ")
        print(f"\nUser: {user_question}")
        if user_question.lower() in ["exit", "quit"]:
            print("\nAgent: Goodbye!")
            break
        
        # Invoke the agent executor with the user's question
        response = agent_executor.invoke({"input": user_question})
        
        # Print the final answer
        print(f"\nAgent: {response['output']}")

    except Exception as e:
        print(f"An error occurred: {e}")

Chat with the Hospital AI Agent! Type 'exit' to quit.

User: how many hospital in Florida?


> Entering new AgentExecutor chain...
I need to find out the number of hospitals in Florida. Since this is an objective question that involves counting, I will use the HospitalGraphQA tool to get the information.

Action: HospitalGraphQA  
Action Input: how many hospital in Florida?  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)
WHERE h.state_name = 'FL'
RETURN COUNT(h) AS hospital_count
Full Context:
[{'hospital_count': 7}]

> Finished chain.
{'query': 'how many hospital in Florida?', 'result': 'There are 7 hospitals in Florida.'}

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


I now know the final answer.  
Final Answer: There are 7 hospitals in Florida.

> Finished chain.

Agent: There are 7 hospitals in Florida.

User: can you name them?


> Entering new AgentExecutor chain...
I need to find out the names of the hospitals in Florida. Since this is an objective question that involves listing facts, I will use the HospitalGraphQA tool to retrieve this information.

Action: HospitalGraphQA  
Action Input: "Can you name the hospitals in Florida?"  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)
WHERE h.state_name = 'FL'
RETURN h.name AS hospital_name
Full Context:
[{'hospital_name': 'Walton LLC'}, {'hospital_name': 'Wheeler, Bryant and Johns'}, {'hospital_name': 'Brown Inc'}, {'hospital_name': 'Smith, Edwards and Obrien'}, {'hospital_name': 'Rose Inc'}, {'hospital_name': 'Jordan Inc'}, {'hospital_name': 'Lewis-Nelson'}]

> Finished chain.
{'query': 'Can you name the hospitals in Florida?', 'result': 'The hospitals in Florida a

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


I now know the final answer.  
Final Answer: The hospitals in Florida are as follows:  
1. Walton LLC  
2. Wheeler, Bryant and Johns  
3. Brown Inc  
4. Smith, Edwards and Obrien  
5. Rose Inc  
6. Jordan Inc  
7. Lewis-Nelson  

> Finished chain.

Agent: The hospitals in Florida are as follows:  
1. Walton LLC  
2. Wheeler, Bryant and Johns  
3. Brown Inc  
4. Smith, Edwards and Obrien  
5. Rose Inc  
6. Jordan Inc  
7. Lewis-Nelson

User: exit

Agent: Goodbye!


In [28]:
from agent_evaluator import evaluate_agent
import pandas as pd

In [29]:
csv_filepath = "../data/evaluation_data.csv"

In [30]:
summary, details_df = evaluate_agent(agent_executor, csv_filepath)

100
Starting evaluation on 100 questions...

[Test 1/100] Question: What is the total number of patients treated at Grace Memorial Hospital in May 2023?


> Entering new AgentExecutor chain...
To find the total number of patients treated at Grace Memorial Hospital in May 2023, I will need to use the HospitalGraphQA tool, as this question involves counting patients, which is an objective query.

Action: HospitalGraphQA  
Action Input: What is the total number of patients treated at Grace Memorial Hospital in May 2023?  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)<-[:AT]-(v:Visit)<-[:HAS]-(p:Patient)
WHERE h.name = 'Grace Memorial Hospital' AND v.admission_date >= '2023-05-01' AND v.admission_date < '2023-06-01'
RETURN COUNT(p) AS total_patients_treated
Full Context:
[{'total_patients_treated': 0}]

> Finished chain.
{'query': 'What is the total number of patients treated at Grace Memorial Hospital in May 2023?', 'result': 'In May 2023, a total of 0 p

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


To find the list of physicians specializing in Cardiology, I will use the HospitalGraphQA tool to retrieve this information.

Action: HospitalGraphQA  
Action Input: List all physicians specializing in Cardiology.  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (phy:Physician)
WHERE phy.school CONTAINS 'Cardiology'
RETURN phy.name AS physician_name, phy.id AS physician_id
Full Context:
[]

> Finished chain.
{'query': 'List all physicians specializing in Cardiology.', 'result': "I don't know the answer."}It seems that the HospitalGraphQA tool did not provide any information regarding the physicians specializing in Cardiology. I will try to gather insights about patient experiences or reviews related to cardiology specialists, which might help in understanding the available physicians in that specialty.

Action: HospitalReviewsQA  
Action Input: Are there any reviews or experiences shared by patients about cardiology specialists?  {'query': 'Are there any reviews or

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


To find the billing amount for visit ID 789123, I need to query the relevant information about hospital visits, which includes billing details.

Action: HospitalGraphQA  
Action Input: What is the billing amount for visit ID 789123?  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Visit)
WHERE v.id = 789123
MATCH (v)-[c:COVERED_BY]->(p:Payer)
RETURN c.billing_amount AS billing_amount
Full Context:
[]

> Finished chain.
{'query': 'What is the billing amount for visit ID 789123?', 'result': "I'm sorry, but I don't know the answer to your question about the billing amount for visit ID 789123."}I was unable to retrieve the billing amount for visit ID 789123, as the information is not available. 

Final Answer: I do not have the billing amount for visit ID 789123.

> Finished chain.
  -> Expected Tool: HospitalGraphQA
  -> Actual Tool:   HospitalGraphQA
  -> Selected At:   2025-06-18 03:55:25
  -> Correct:       True
  -> Latency:       5.31s

[Test 4/100] Question: 

...

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


To answer the question about the other hospitals that Jane Doe's primary care physician is affiliated with, I need to gather information about the physician's affiliations. Since I don't have that information directly, I will use the HospitalGraphQA tool to find out the affiliations of the physician.

Action: HospitalGraphQA  
Action Input: "What other hospitals is Jane Doe's primary care physician affiliated with?"  

> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Patient {name: 'Jane Doe'})-[:HAS]->(v:Visit)<-[:TREATS]-(phy:Physician)<-[:EMPLOYS]-(h:Hospital)
WITH phy, h
MATCH (phy)<-[:EMPLOYS]-(h2:Hospital)
WHERE h <> h2
RETURN h2.name AS affiliated_hospital
Full Context:
[]

> Finished chain.
{'query': "What other hospitals is Jane Doe's primary care physician affiliated with?", 'result': "I don't know the answer."}I still do not have the information regarding the affiliations of Jane Doe's primary care physician. 

Final Answer: I don't know the answer.

> F

/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


Action: None  
Action Input: None  None is not a valid tool, try one of [HospitalGraphQA, HospitalReviewsQA].Action: None  
Action Input: None  None is not a valid tool, try one of [HospitalGraphQA, HospitalReviewsQA].Action: None  
Action Input: None  None is not a valid tool, try one of [HospitalGraphQA, HospitalReviewsQA].Action: None  
Action Input: None  None is not a valid tool, try one of [HospitalGraphQA, HospitalReviewsQA].I cannot proceed with the current question as it does not require an action or information retrieval. 

Final Answer: No further action is needed at this time.

> Finished chain.
  -> Expected Tool: no
  -> Actual Tool:   None
  -> Selected At:   2025-06-18 04:04:03
  -> Correct:       False
  -> Latency:       3.39s

--- Evaluation Complete ---


/Users/vansy/Documents/Projects/vansy-f8a-genai-test/.venv/lib/python3.13/site-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferWindowMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


In [31]:
if summary:
    print("\n\n📊 EVALUATION SUMMARY:")
    for key, value in summary.items():
        if isinstance(value, float):
            print(f"- {key.replace('_', ' ').title()}: {value:.4f}")
        else:
            print(f"- {key.replace('_', ' ').title()}: {value}")

    print("\n\n📋 DETAILED RESULTS (PREVIEW):")
    display(details_df)

    # NEW: Save the detailed results DataFrame to a new CSV file
    output_filename = "../data/evaluation_results.csv"
    details_df.to_csv(output_filename, index=False)
    print(f"\n\n💾 Detailed results saved to '{output_filename}'")



📊 EVALUATION SUMMARY:
- Total Questions: 100
- Total Cost Usd: 0.0133
- Total Tokens: 96648
- Average Latency Sec: 5.4780
- Tool Selection Accuracy Percent: 85.4167


📋 DETAILED RESULTS (PREVIEW):


,num,question,expected_tool,actual_tool,tool_selection_datetime,is_correct,latency_sec,final_answer
0,0,What is the total number of patients treated a...,HospitalGraphQA,HospitalGraphQA,2025-06-18 03:55:10,True,6.142811,"In May 2023, a total of 0 patients were treate..."
1,1,List all physicians specializing in Cardiology.,HospitalGraphQA,HospitalGraphQA,2025-06-18 03:55:20,True,10.071405,I am unable to provide a list of physicians sp...
2,2,What is the billing amount for visit ID 789123?,HospitalGraphQA,HospitalGraphQA,2025-06-18 03:55:25,True,5.310497,I do not have the billing amount for visit ID ...
3,3,Which insurance provider is associated with pa...,HospitalGraphQA,HospitalGraphQA,2025-06-18 03:55:30,True,4.394001,I do not have information about the insurance ...
4,5,Find all patients treated by Dr. Ben Adams.,HospitalGraphQA,HospitalGraphQA,2025-06-18 03:55:37,True,3.901840,I don't know the answer.
...,...,...,...,...,...,...,...,...
91,95,Now show me the total number of visits for tha...,HospitalGraphQA,HospitalGraphQA,2025-06-18 04:03:36,True,4.329020,The total number of visits for Downtown Clinic...
92,96,Who is patient Jane Doe's primary care physician?,HospitalGraphQA,HospitalGraphQA,2025-06-18 04:03:40,True,4.278466,I don't know the answer.
93,97,What do other patients say about him?,HospitalReviewsQA,HospitalGraphQA,2025-06-18 04:03:55,False,14.700387,"Based on the patient reviews provided, here ar..."
94,98,And what other hospitals is he affiliated with?,HospitalGraphQA,HospitalGraphQA,2025-06-18 04:04:00,True,4.773778,I don't know the answer.




💾 Detailed results saved to '../data/evaluation_results.csv'


In [36]:
details_df.dtypes

num                          int64
question                    object
expected_tool               object
actual_tool                 object
tool_selection_datetime     object
is_correct                    bool
latency_sec                float64
final_answer                object
dtype: object